# **📝 UTS Penambangan Data - Klasifikasi Prediksi Survival Pasien Sirosis**



**👤 Nama: Muhammad Hanif**

**🧪 Mata Kuliah: Penambangan Data**

**🗓️ Proyek: Ujian Tengah Semester (UTS)**

## **Install Library**
Install Library yang diperlukan

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# **1. Mengambil file data**
Kode ini digunakan untuk mengunggah dan memuat file CSV yang berisi data pasien sirosis hati ke dalam Google Colab. Pertama, modul `files` dari `google.colab` diimpor untuk memungkinkan pengguna mengunggah file dari perangkat lokal. Setelah itu, file CSV yang diunggah dimuat ke dalam objek DataFrame menggunakan `pandas` dengan fungsi `pd.read_csv()`. Data yang dimuat kemudian ditampilkan lima baris pertama menggunakan `df.head()` untuk memberikan gambaran awal mengenai struktur data, termasuk nama-nama kolom dan nilai-nilai yang terdapat pada baris pertama. Langkah-langkah ini termasuk dalam tahap **Pemahaman Data** yang penting dalam analisis data untuk mengeksplorasi dan memeriksa data sebelum dilakukan pemrosesan lebih lanjut.

In [ ]:
from google.colab import files

# Unggah file dari lokal
uploaded = files.upload()

# Setelah file diunggah, kamu bisa memuatnya dengan pandas
import pandas as pd

# Nama file yang diunggah adalah 'cirrhosis.csv'
df = pd.read_csv('cirrhosis.csv')

# Lihat beberapa baris pertama
print(df.head())

Saving cirrhosis.csv to cirrhosis.csv
   ID  N_Days Status             Drug    Age Sex Ascites Hepatomegaly Spiders  \
0   1     400      D  D-penicillamine  21464   F       Y            Y       Y   
1   2    4500      C  D-penicillamine  20617   F       N            Y       Y   
2   3    1012      D  D-penicillamine  25594   M       N            N       N   
3   4    1925      D  D-penicillamine  19994   F       N            Y       Y   
4   5    1504     CL          Placebo  13918   F       N            Y       Y   

  Edema  Bilirubin  Cholesterol  Albumin  Copper  Alk_Phos    SGOT  \
0     Y       14.5        261.0     2.60   156.0    1718.0  137.95   
1     N        1.1        302.0     4.14    54.0    7394.8  113.52   
2     S        1.4        176.0     3.48   210.0     516.0   96.10   
3     S        1.8        244.0     2.54    64.0    6121.8   60.63   
4     N        3.4        279.0     3.53   143.0     671.0  113.15   

   Tryglicerides  Platelets  Prothrombin  Stage  
0   

# **2. Load Data**
Bagian ini bertujuan untuk membaca file CSV yang berisi data pasien sirosis hati menggunakan pd.read_csv. Setelah data dimuat, df.shape digunakan untuk menampilkan ukuran data (jumlah baris dan kolom). df.head() digunakan untuk menampilkan beberapa baris pertama data, agar dapat memeriksa data awal secara sekilas.

In [ ]:
df = pd.read_csv("cirrhosis.csv")
print("Data awal:", df.shape)
print(df.head())

Data awal: (418, 20)
   ID  N_Days Status             Drug    Age Sex Ascites Hepatomegaly Spiders  \
0   1     400      D  D-penicillamine  21464   F       Y            Y       Y   
1   2    4500      C  D-penicillamine  20617   F       N            Y       Y   
2   3    1012      D  D-penicillamine  25594   M       N            N       N   
3   4    1925      D  D-penicillamine  19994   F       N            Y       Y   
4   5    1504     CL          Placebo  13918   F       N            Y       Y   

  Edema  Bilirubin  Cholesterol  Albumin  Copper  Alk_Phos    SGOT  \
0     Y       14.5        261.0     2.60   156.0    1718.0  137.95   
1     N        1.1        302.0     4.14    54.0    7394.8  113.52   
2     S        1.4        176.0     3.48   210.0     516.0   96.10   
3     S        1.8        244.0     2.54    64.0    6121.8   60.63   
4     N        3.4        279.0     3.53   143.0     671.0  113.15   

   Tryglicerides  Platelets  Prothrombin  Stage  
0          172.0     

# **3. Drop Kolom ID**
Di sini, kolom ID dihapus karena kolom ini tidak relevan untuk analisis atau model yang akan dibangun. inplace=True berarti perubahan ini langsung diterapkan pada data df tanpa membuat salinan baru.

In [ ]:
df.drop(columns=['ID'], inplace=True)

## **4. Mapping Kolom Status**
Pada langkah ini, data difilter untuk memastikan bahwa hanya kategori yang valid dari Status yang ada, yaitu 'C', 'D', dan 'CL'. Kemudian, kolom Status diubah menggunakan .map(), di mana:

'D' (Died) diubah menjadi 1 (positif).

'C' (Censored) dan 'CL' (Censored for liver) diubah menjadi 0 (negatif).

Dengan demikian, target yang akan diprediksi hanya terdiri dari dua kelas: 1 (Died) dan 0 (Censored).



In [ ]:
df = df[df['Status'].isin(['C', 'D', 'CL'])]  # Pastikan hanya 3 kategori
df['Status'] = df['Status'].map({'D': 1, 'C': 0, 'CL': 0})

## **5. Encode Kolom Kategorikal**
Bagian ini melakukan encoding pada beberapa kolom kategorikal menjadi nilai numerik:

Kolom Ascites, Hepatomegaly, dan Spiders diubah menjadi 1 untuk 'Y' dan 0 untuk 'N'.

Kolom Edema juga diubah menjadi 1 untuk 'Y', 0 untuk 'N', dan 0.5 untuk 'S' (ini mungkin menggambarkan kasus yang tidak pasti).

Kolom Sex diubah menjadi 1 untuk 'M' (Male) dan 0 untuk 'F' (Female).

Kolom Drug yang berisi kategori obat, diubah menjadi kode numerik menggunakan astype('category').cat.codes yang secara otomatis mengonversi semua kategori ke dalam angka.

In [ ]:
df['Ascites'] = df['Ascites'].map({'Y': 1, 'N': 0})
df['Hepatomegaly'] = df['Hepatomegaly'].map({'Y': 1, 'N': 0})
df['Spiders'] = df['Spiders'].map({'Y': 1, 'N': 0})
df['Edema'] = df['Edema'].map({'Y': 1, 'N': 0, 'S': 0.5})

df['Sex'] = df['Sex'].map({'M': 1, 'F': 0})

df['Drug'] = df['Drug'].astype('category').cat.codes  # Auto encode semua kategori


## **6. Isi Missing Values dengan Median**
Di sini, missing values (NaN) yang ada pada kolom numerik diisi dengan nilai median dari kolom tersebut. Ini membantu menghindari masalah yang mungkin timbul jika ada data yang hilang pada fitur numerik, tanpa harus membuang baris yang memiliki nilai kosong.

In [ ]:
df = df.fillna(df.median(numeric_only=True))

## **7. Drop Baris yang Masih Ada NaN**
Pada langkah ini, semua baris yang masih mengandung nilai NaN (meskipun setelah dilakukan imputasi) akan dihapus. Ini memastikan bahwa data yang akan digunakan untuk pelatihan dan pengujian benar-benar lengkap.

In [ ]:
df = df.dropna()

## **8. Pisahkan Fitur dan Target**
Di sini, data dipisahkan menjadi dua bagian:

X adalah fitur (input) yang digunakan untuk memprediksi status. Semua kolom kecuali Status dipilih sebagai fitur.

y adalah target (output) yang ingin diprediksi, yaitu kolom Status.

In [ ]:
X = df.drop(columns='Status')
y = df['Status']

## **9. Split Data**
Data dibagi menjadi dua bagian:

Training set (X_train dan y_train) digunakan untuk melatih model.

Test set (X_test dan y_test) digunakan untuk menguji model dan mengevaluasi kinerjanya.

Pembagian data dilakukan dengan menggunakan train_test_split dari sklearn.model_selection, di mana 20% data digunakan untuk pengujian (test_size=0.2), dan 80% untuk pelatihan.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## **10. Model dan Pelatihan**
Di sini, sebuah model Decision Tree dibuat menggunakan DecisionTreeClassifier dari sklearn.tree. Model ini dilatih menggunakan data pelatihan (X_train, y_train) dengan parameter random_state=42 untuk memastikan hasil yang konsisten setiap kali kode dijalankan.

In [ ]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

## **11. Evaluasi**
Setelah model dilatih, digunakan untuk memprediksi hasil pada data uji (X_test). Hasil prediksi (y_pred) dibandingkan dengan nilai aktual (y_test) untuk mengevaluasi performa model:

Akurasi dihitung dengan accuracy_score, yang menunjukkan seberapa banyak prediksi yang benar.

Laporan klasifikasi menggunakan classification_report, yang memberikan metrik evaluasi seperti precision, recall, f1-score, dan support untuk setiap kelas (Censored dan Died).

In [ ]:
y_pred = model.predict(X_test)
print("\nAkurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan klasifikasi:\n", classification_report(y_test, y_pred))


Akurasi: 0.6785714285714286

Laporan klasifikasi:
               precision    recall  f1-score   support

           0       0.74      0.67      0.70        48
           1       0.61      0.69      0.65        36

    accuracy                           0.68        84
   macro avg       0.68      0.68      0.68        84
weighted avg       0.69      0.68      0.68        84



## **Hasil analisis lengkap**
Berikut adalah kode lengkapnya :

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Load data
df = pd.read_csv("cirrhosis.csv")
print("Data awal:", df.shape)
print(df.head())

# 2. Drop kolom ID
df.drop(columns=['ID'], inplace=True)

# 3. Mapping kolom Status dengan benar
df = df[df['Status'].isin(['C', 'D', 'CL'])]  # Pastikan hanya 3 kategori
df['Status'] = df['Status'].map({'D': 1, 'C': 0, 'CL': 0})

# 4. Encode kolom kategorikal
df['Ascites'] = df['Ascites'].map({'Y': 1, 'N': 0})
df['Hepatomegaly'] = df['Hepatomegaly'].map({'Y': 1, 'N': 0})
df['Spiders'] = df['Spiders'].map({'Y': 1, 'N': 0})
df['Edema'] = df['Edema'].map({'Y': 1, 'N': 0, 'S': 0.5})

df['Sex'] = df['Sex'].map({'M': 1, 'F': 0})

# Mapping 'Drug' → encode semua yang ada
df['Drug'] = df['Drug'].astype('category').cat.codes  # Auto encode semua kategori

# 5. Isi missing values dengan median
df = df.fillna(df.median(numeric_only=True))

# 6. Drop baris yang masih ada NaN (misal karena kategori aneh)
df = df.dropna()

print("Data setelah preprocessing:", df.shape)

# 7. Pisahkan fitur dan target
X = df.drop(columns='Status')
y = df['Status']

# 8. Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 9. Model & Pelatihan
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# 10. Evaluasi
y_pred = model.predict(X_test)
print("\nAkurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan klasifikasi:\n", classification_report(y_test, y_pred))

Data awal: (418, 20)
   ID  N_Days Status             Drug    Age Sex Ascites Hepatomegaly Spiders  \
0   1     400      D  D-penicillamine  21464   F       Y            Y       Y   
1   2    4500      C  D-penicillamine  20617   F       N            Y       Y   
2   3    1012      D  D-penicillamine  25594   M       N            N       N   
3   4    1925      D  D-penicillamine  19994   F       N            Y       Y   
4   5    1504     CL          Placebo  13918   F       N            Y       Y   

  Edema  Bilirubin  Cholesterol  Albumin  Copper  Alk_Phos    SGOT  \
0     Y       14.5        261.0     2.60   156.0    1718.0  137.95   
1     N        1.1        302.0     4.14    54.0    7394.8  113.52   
2     S        1.4        176.0     3.48   210.0     516.0   96.10   
3     S        1.8        244.0     2.54    64.0    6121.8   60.63   
4     N        3.4        279.0     3.53   143.0     671.0  113.15   

   Tryglicerides  Platelets  Prothrombin  Stage  
0          172.0     